# **PREPROCESAMIENTO DE IMÁGENES**

In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

_____________
* 1. Verificar y normalizar rutas a archivos .nii

In [2]:
# [Config] Rutas
# Ajusta estas rutas según tu estructura
BASE_IMAGES_DIR = Path(r"C:\Users\Hp\MACHINE\MRI\Datos\IMAGES")   # ya usado en df['ruta'] (puede ser relativo)
NOTEBOOKS_DIR = Path(r"C:\Users\Hp\MACHINE\MRI\notebooks")
OUT_DIR = Path(r"C:\Users\Hp\MACHINE\MRI\datos\procesadas")      # donde dejaremos los .npy después
OUT_DIR.mkdir(parents=True, exist_ok=True)
df = pd.read_csv("../Datos/Clinical/ADNI_Images.csv")

In [37]:
df.shape

(283, 44)

In [ ]:
# Normalizar rutas a los archivos nii
def normalize_path(p):
    # convierte rutas relativas con .. y / a una Path absoluta
    p = str(p)
    # Reemplaza barras si vienen con mayúsculas, espacios extra, etc.
    return Path(p.replace("/", os.sep)).resolve()
df['ruta_abs'] = df['ruta'].apply(lambda p: normalize_path(p) if pd.notnull(p) else p)
df['archivo'] = df['archivo'].astype(str)

________
* 2. Crear identificadores y nombres de salida

In [ ]:
# Normalizamos campos clave
df['sujeto_id'] = df['sujeto_id'].astype(str)
df['VISCODE'] = df['VISCODE'].astype(str)
# EXAMDATE -> intentar parsear (si ya es datetime, no cambia)
def parse_date(x):
    if pd.isnull(x): 
        return pd.NaT
    if isinstance(x, (datetime, pd.Timestamp)):
        return pd.to_datetime(x)
    # probar distintos formatos comunes
    for fmt in ("%Y-%m-%d", "%Y/%m/%d", "%d-%m-%Y", "%Y-%m-%d %H:%M:%S", "%Y-%m-%dT%H:%M:%S"):
        try:
            return pd.to_datetime(x, format=fmt)
        except Exception:
            pass
    return pd.to_datetime(x, errors='coerce')

df['EXAMDATE_parsed'] = df['EXAMDATE'].apply(parse_date)

Fechas no parseadas: 0


In [ ]:
# id_visita = sujetoid_VISCODE
df['id_visita'] = df['sujeto_id'] + "_" + df['VISCODE']
# fname_out (sin subcarpetas): sujetoid_EXAMDATE_VISCODE_archivoSinExt.npy
def safe_fname(s):
    # quita espacios, reemplaza / : etc
    return "".join(c if c.isalnum() or c in ("-", "_") else "_" for c in s)

def make_output_fname(row):
    subj = row['sujeto_id']
    vis = row['VISCODE']
    examdate = row['EXAMDATE_parsed']
    exam_str = examdate.strftime("%Y%m%d") if pd.notnull(examdate) else "noDate"
    arch = Path(row['archivo']).stem
    fname = f"{subj}_{exam_str}_{vis}_{arch}.npy"
    return safe_fname(fname)

df['fname_out'] = df.apply(make_output_fname, axis=1)
df['ruta_npy'] = df['fname_out'].apply(lambda s: OUT_DIR / s)


In [19]:
# Guardamos manifest preliminar (sin procesar imágenes aún)
manifest_pre = df[['sujeto_id','id_visita','VISCODE','EXAMDATE','EXAMDATE_parsed','archivo','ruta','ruta_abs','ruta_npy','DX']]
manifest_pre.to_csv(OUT_DIR / "manifest_preprocesamiento.csv", index=False)
print("Manifest preliminar guardado en:", OUT_DIR / "manifest_preprocesamiento.csv")


Manifest preliminar guardado en: C:\Users\Hp\MACHINE\MRI\datos\procesadas\manifest_preprocesamiento.csv


____________
* 3. Construir etiquetas binarias

In [ ]:
# Definimos mapa VISCODE -> meses
viscode_to_month = {'bl':0, 'm06':6, 'm12':12, 'm18':18, 'm24':24, 'm30':30, 'm36':36}
df['vis_month'] = df['VISCODE'].map(viscode_to_month)



array([ 0,  6, 12, 18, 24, 30, 36], dtype=int64)

In [32]:
def dx_is_dementia(dx):
    if pd.isnull(dx):
        return np.nan
    s = str(dx).lower()
    if 'dementia' in s:
        return 1
    if 'mci' in s:
        return 0
    return np.nan
df['is_dementia'] = df['DX'].apply(dx_is_dementia)

In [35]:
# Construir label por sujeto:
# Regla: si sujeto presenta transición de non-dementia -> dementia en algún vis <=36 meses -> label 1.
# Implementación simplificada:
labels = []
subjects = df['sujeto_id'].unique()
for subj in tqdm(subjects, desc="Calculando labels por sujeto"):
    subdf = df[df['sujeto_id']==subj].copy()
    # orden por vis_month (si NaN, orden por EXAMDATE_parsed)
    if subdf['vis_month'].notna().any():
        subdf = subdf.sort_values(by=['vis_month','EXAMDATE_parsed'])
    else:
        subdf = subdf.sort_values(by='EXAMDATE_parsed')
    # buscamos primer visita con is_dementia==1 y su mes
    # y primer visita con is_dementia==0 (baseline non-dementia)
    baseline_non_d = subdf[subdf['is_dementia']==0]
    first_non_d = None
    if not baseline_non_d.empty:
        # preferir VISCODE 'bl' si existe
        if (baseline_non_d['VISCODE'].str.lower()=='bl').any():
            first_non_d = baseline_non_d[baseline_non_d['VISCODE'].str.lower()=='bl'].iloc[0]
        else:
            first_non_d = baseline_non_d.iloc[0]
    # buscar conversion
    converted = False
    conversion_info = None
    # consideramos cualquier visita con is_dementia==1 y vis_month <= 36
    dem_vis = subdf[subdf['is_dementia']==1]
    if not dem_vis.empty:
        # si hay baseline non-dementia, la conversión debe ocurrir tras baseline
        if first_non_d is not None:
            # buscar dem_vis cuya month >= baseline month y <=36
            base_month = first_non_d.get('vis_month', 0) if pd.notnull(first_non_d.get('vis_month', np.nan)) else 0
            # candidate dem_vis posteriores
            for _, r in dem_vis.iterrows():
                month = r.get('vis_month', np.nan)
                if pd.isna(month):
                    # si no hay month, intentar por fecha diff
                    if pd.notnull(first_non_d['EXAMDATE_parsed']) and pd.notnull(r['EXAMDATE_parsed']):
                        diff_months = (r['EXAMDATE_parsed'].year - first_non_d['EXAMDATE_parsed'].year)*12 + (r['EXAMDATE_parsed'].month - first_non_d['EXAMDATE_parsed'].month)
                        month = diff_months
                # now check range
                if pd.notnull(month) and (month >= base_month) and (month <= 36):
                    converted = True
                    conversion_info = (r['VISCODE'], month, r['EXAMDATE_parsed'])
                    break
        else:
            # No baseline non-dementia conocido: fallback -> if any dem_vis within <=36 -> label may be ambiguous:
            # we'll mark converted if there's any dem_vis with vis_month<=36
            for _, r in dem_vis.iterrows():
                month = r.get('vis_month', np.nan)
                if pd.notnull(month) and month <= 36:
                    converted = True
                    conversion_info = (r['VISCODE'], month, r['EXAMDATE_parsed'])
                    break
    # Save label for all rows of this subject (we attach label at image-row level later)
    labels.append({'sujeto_id': subj, 'label_progresion': int(converted), 'conversion_info': conversion_info})

labels_df = pd.DataFrame(labels)
print("Labels por sujeto (ejemplo):")
display(labels_df.head(10))

Calculando labels por sujeto: 100%|██████████| 53/53 [00:00<00:00, 282.59it/s]

Labels por sujeto (ejemplo):


,sujeto_id,label_progresion,conversion_info
0,007_S_0101,1,"(m24, 24, 2007-12-12 00:00:00)"
1,007_S_0128,1,"(m18, 18, 2007-08-20 00:00:00)"
2,007_S_0249,1,"(m12, 12, 2007-04-03 00:00:00)"
3,013_S_0240,1,"(m18, 18, 2007-10-31 00:00:00)"
4,014_S_0169,0,None
5,018_S_0057,1,"(m18, 18, 2007-08-21 00:00:00)"
6,018_S_0080,0,None
7,018_S_0087,0,None
8,018_S_0103,0,None
9,018_S_0142,0,None


In [36]:
# Merge back al df (por sujeto_id)
df = df.merge(labels_df[['sujeto_id','label_progresion']], on='sujeto_id', how='left')

# Guardar manifest con label
manifest_final = df[['sujeto_id','id_visita','VISCODE','EXAMDATE','EXAMDATE_parsed','archivo','ruta_abs','ruta_npy','DX','is_dementia','label_progresion']]
manifest_final.to_csv(OUT_DIR / "manifest_labels.csv", index=False)
print("Manifest final con labels guardado en:", OUT_DIR / "manifest_labels.csv")

# Resumen de labels
print("Resumen labels (por sujeto):")
display(labels_df['label_progresion'].value_counts(dropna=False))

# Guardamos también un CSV de sujetos con label para inspección
labels_df.to_csv(OUT_DIR / "labels_por_sujeto.csv", index=False)
print("CSV labels por sujeto guardado en:", OUT_DIR / "labels_por_sujeto.csv")

Manifest final con labels guardado en: C:\Users\Hp\MACHINE\MRI\datos\procesadas\manifest_labels.csv
Resumen labels (por sujeto):


label_progresion
0    31
1    22
Name: count, dtype: int64

CSV labels por sujeto guardado en: C:\Users\Hp\MACHINE\MRI\datos\procesadas\labels_por_sujeto.csv
